# 데이터셋 로드

In [9]:
from datasets import load_dataset

dataset = load_dataset("KorQuAD/squad_kor_v1")

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [17]:
import pandas as pd

df_train = pd.DataFrame(dataset['train'])

In [ ]:
df_valid = pd.DataFrame(dataset['validation'])

# 1차 필터링 (LLM으로 context만 판별)

## train dataset에서 유효 id를 가진 row만 추출
train dataset: 60407 -> 9041

In [18]:
df = pd.read_csv('korquad_train_cls.csv')
df = df.drop(columns=['Unnamed: 0'])
df_ok = df[df['cls']=='유효']
df_ok_ids = df_ok['id'].to_list()

In [23]:
# 리스트에서 앞의 두 구간 추출
id_prefix_set = {f"{id.split('-')[0]}-{id.split('-')[1]}" for id in df_ok_ids}

# DataFrame에서 동일한 패턴 추출
df_train['prefix'] = df_train['id'].apply(lambda x: f"{x.split('-')[0]}-{x.split('-')[1]}")

# 필터링
filtered_df = df_train[df_train['prefix'].isin(id_prefix_set)].drop(columns=['prefix'])

filtered_df.to_csv('korquad_train_remain.csv')

## valid dataset에서 유효 id를 가진 row만 추출
valid dataset: 5774 -> 898

In [6]:
df = pd.read_csv('korquad_valid_cls.csv')
df = df.drop(columns=['Unnamed: 0'])
df_ok = df[df['cls']=='유효']

In [7]:
df_ok_ids = df_ok['id'].to_list()

In [16]:
# 리스트에서 앞의 두 구간 추출
id_prefix_set = {f"{id.split('-')[0]}-{id.split('-')[1]}" for id in df_ok_ids}

# DataFrame에서 동일한 패턴 추출
df_valid['prefix'] = df_valid['id'].apply(lambda x: f"{x.split('-')[0]}-{x.split('-')[1]}")

# 필터링
filtered_df = df_valid[df_valid['prefix'].isin(id_prefix_set)].drop(columns=['prefix'])

filtered_df.to_csv('korquad_valid_remain.csv')

## concat

In [25]:
df_train_reamain = pd.read_csv('korquad_train_remain.csv')
df_valid_reamain = pd.read_csv('korquad_valid_remain.csv')

In [28]:
df_reamin = pd.concat([df_train_reamain, df_valid_reamain])
df_reamin = df_reamin.drop(columns=['Unnamed: 0'])

In [30]:
df_reamin.to_csv('korquad_remain.csv')

# context 중복 제거해가면서 만들어보자
(보류: 2차 필터링 (LLM으로 context와 question, answer set 판별))

In [9]:
import pandas as pd

df_remain = pd.read_csv('korquad_remain.csv')
df_remain = df_remain.drop(columns=['Unnamed: 0'])

In [10]:
df_remain_first = df_remain.drop_duplicates(subset='context', keep='first')

In [ ]:
df_remain_first.to_csv('df_remain_first.csv')

In [11]:
# 중복으로 걸러진 데이터 가져오기
df_filtered_out = df_remain[~df_remain.index.isin(df_remain_first.index)]

In [13]:
print(len(df_remain))
print(len(df_remain_first))
print(len(df_filtered_out))

9939
4186
5753


In [14]:
df_remain_second = df_filtered_out.drop_duplicates(subset='context', keep='first')

In [16]:
df_remain_second.to_csv('df_remain_second.csv')

In [17]:
# 중복으로 걸러진 데이터 가져오기
df_filtered_out_again = df_filtered_out[~df_filtered_out.index.isin(df_remain_second.index)]

In [18]:
print(len(df_filtered_out))
print(len(df_remain_second))
print(len(df_filtered_out_again))

5753
4184
1569


In [19]:
df_filtered_out_again.to_csv('df_remain_third.csv')

### generate test

In [1]:
import pandas as pd

df= pd.read_csv('df_remain_third.csv')

In [3]:
df_head5 = df.iloc[:5]

In [4]:
df_head5.to_csv('head5_test.csv')